In [1]:
import os 
import numpy as np 
import h5py 
import json
import shutil
import time 
import torch 
import glob 
import scipy.io as sio

from tqdm import tqdm
from datetime import datetime
from utils import *

# torch computation device to calculate jacobian via autograd
device = torch.device('cuda:0')

/home/nikolai/dev/constrained_inversion_mrm/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
opt_pulse_directory = 'data/nonsel_20250313_072103'
opt_pulse_file = os.path.join(opt_pulse_directory, 'pulse_000.h5')
opt_pulse_param_file = os.path.join(opt_pulse_directory, 'params_nonsel.json')

with open(opt_pulse_param_file,'r') as fid: par = json.load(fid)[0]
gamma_bar = par['gamma_bar']
dt = par['raster_time']

# load optimized pulse and reference HS pulse
with h5py.File(opt_pulse_file, 'r') as F:
    rf_opt = np.array(F['rf_opt'], dtype=np.complex128) * 1e6 / gamma_bar # uT
    rf_hs = np.array(F['rf_hs'], dtype=np.complex128) * 1e6 / gamma_bar   # uT
    mz_hs = np.array(F['mz_hs'], dtype=np.float64)
    mz_opt = np.array(F['mz_opt'], dtype=np.float64)
    b1_design = np.array(F['b1_scales'], dtype=np.float64) 
    freq_design = np.array(F['freq_arr'], dtype=np.float64)
dur_opt = rf_opt.size * dt 
dur_hs = rf_hs.size * dt 

# load the pulse provided by Graf et al
graf_pulse_file = 'data/pulse_graf_et_al.mat'
df = sio.loadmat(graf_pulse_file)
u = np.squeeze(df['u']).astype(np.float64)
v = np.squeeze(df['v']).astype(np.float64)
rf_graf = (u + 1j*v) * 1e3 # uT
dt_graf = 0.01*1e-3
dur_graf = rf_graf.size * dt_graf

freq_eval = np.linspace(-1000.0, 1000.0, 512, dtype=np.float64)
b1_eval = np.linspace(0.0, 2.0, 512, dtype=np.float64)

# simulations of longitudinal magnetization
mz_graf = simulate_mz(rf_graf*1e-6*gamma_bar, freq_design, b1_design, dt_graf, device=device, return_numpy=True)
mz_graf_eval = simulate_mz(rf_graf*1e-6*gamma_bar, freq_eval, b1_eval, dt_graf, device=device, return_numpy=True)
mz_opt_eval = simulate_mz(rf_opt*1e-6*gamma_bar, freq_eval, b1_eval, dt, device=device, return_numpy=True)
mz_hs_eval = simulate_mz(rf_hs*1e-6*gamma_bar, freq_eval, b1_eval, dt, device=device, return_numpy=True)

# time vectors to be used for plotting 
tv_opt = 1e3 * np.arange(0, dur_opt, dt, dtype=np.float64)
tv_hs = 1e3 * np.arange(0, dur_hs, dt, dtype=np.float64)
tv_graf = 1e3 * np.arange(0, dur_graf, dt_graf, dtype=np.float64)

pulses = [rf_hs, rf_opt, rf_graf]
tv = [tv_hs, tv_opt, tv_graf]
dur = [dur_hs, dur_opt, dur_graf] # scalar
mz = [mz_hs, mz_opt, mz_graf]
mz_eval = [mz_hs_eval, mz_opt_eval, mz_graf_eval]
show_optimized_region = [0, 1, 1]
x_corner = [0, freq_design.min(), 0.0]
y_corner = [0, b1_design.min(), 0.7] # value of 0.7 hard-coded from Graf et. al.
width = [0, freq_design.max()-freq_design.min(), 0.0]
height = [0, b1_design.max()-b1_design.min(), 1.3 - 0.7]
legend_locs = ['upper right', 'center', 'lower right']
titles = ['$\mathrm{HS\ Reference}$', '$\mathrm{Proposed}$', '$\mathrm{Graf\ et.\ al.\ (optim\ }B_1^+\mathrm{)}$']
num_pulses = len(pulses)

eval_file = os.path.join(opt_pulse_directory, 'eval_data.h5')
with h5py.File(eval_file,'w') as F:
    F.create_dataset('num_pulses', data=num_pulses)
    F.create_dataset('b1_design', data=b1_design)
    F.create_dataset('freq_design', data=freq_design)
    F.create_dataset('b1_eval', data=b1_eval)
    F.create_dataset('freq_eval', data=freq_eval)
    for p in range(len(pulses)):
        F.create_dataset('pulse_%i'%(p), data=pulses[p]) 
        F.create_dataset('tv_%i'%(p), data=tv[p])
        F.create_dataset('dur_%i'%(p), data=dur[p]*1e3)
        F.create_dataset('mz_%i'%(p), data=mz[p])
        F.create_dataset('mz_eval_%i'%(p), data=mz_eval[p])
        F.create_dataset('show_optimized_region_%i'%(p), data=show_optimized_region[p]) 
        F.create_dataset('x_corner_%i'%(p), data=x_corner[p]) 
        F.create_dataset('y_corner_%i'%(p), data=y_corner[p]) 
        F.create_dataset('width_%i'%(p), data=width[p]) 
        F.create_dataset('height_%i'%(p), data=height[p]) 
        F.create_dataset('title_%d'%(p), data=titles[p], dtype=h5py.string_dtype(encoding='utf-8'))
        F.create_dataset('legend_loc_%d'%(p), data=legend_locs[p], dtype=h5py.string_dtype(encoding='utf-8'))

